In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
from PIL import Image
import os
from video_utils import *
from model import MODEL
from tqdm import tqdm
from PIL import Image

In [3]:
#model = YOLO("yolov8l.pt")
# model = MODEL("yolov8l.pt", device = 0)
dataset_root_dir  = "basketballAndHoop-2/train"
reID_dataset_root_dir = "reID_dataset"
train_images = os.listdir(dataset_root_dir + "/images")

# for img_path in train_images:
#     print(img_path)
#     short_name = img_path.split("_jpg.rf.")[0] 
#     full_img_path = dataset_root_dir + "/images/" + img_path
#     full_label_path = dataset_root_dir + "/labels/" + img_path.replace(".jpg", ".txt")
    
#     renamed_img_path = dataset_root_dir + "/images/" + short_name + ".jpg"
#     renamed_label_path = dataset_root_dir + "/labels/" + short_name + ".txt"
    
#     os.rename(full_img_path, renamed_img_path)
#     os.rename(full_label_path, renamed_label_path)


all_environments = list({x[:-7] for x in train_images})
        
# select randomly a subset of environments
n_samples = 30
selected_envs = np.random.choice(all_environments, n_samples, replace=False)
selected_images = {env: [img for img in train_images if env in img] for env in selected_envs}



for env in selected_envs:
    this_dir = reID_dataset_root_dir + "/" + env
    os.makedirs(this_dir, exist_ok=True)
    images = selected_images[env]
    labels = [img.replace(".jpg", ".txt") for img in images]
    for img, label in zip(images, labels):
        img = cv2.imread(dataset_root_dir + "/images/" + img)
        
        person_count = 0
        with open(dataset_root_dir + "/labels/" + label, "r") as f:
            lines = f.readlines()
            for line in lines:
                box = list(map(float, line.split()))
                
                if box and box[0] == 2:
                    x, y, w, h = box[1:]
                    x1, y1 = int((x - w/2) * img.shape[1]), int((y - h/2) * img.shape[0])
                    x2, y2 = int((x + w/2) * img.shape[1]), int((y + h/2) * img.shape[0])
                
                    cropped_person = img[y1:y2, x1:x2]
                    cropped_person_img_path = this_dir + "/" + str(person_count) + ".jpg"
                    cv2.imwrite(cropped_person_img_path, cropped_person)
                    
                    person_count += 1
                
                

In [52]:
person_count

0

In [20]:
im = cv2.imread(dataset_root_dir + "/images/" + selected_images[0])